# MACHINE LEARNING IN GRAPH & TEXT DATA

# LINK PREDICTION IN GRAPH


## S. Calcagno, I. Koval, B. Rozière

This notebook is part of the report of the Machine Learning in graph and text data. The goal was to predict citations between articles based on their title, abstract, authors and year of publication.

The notebook is structured as follow:
- I. DATA IMPORTATION
- II. FEATURE ENGINEERING
- III. CLASSIFIER
- IV. ASSEMBLING
- V. SUBMISSION


### Comments
The code was fully optimized so that it takes less than 45 minutes to create all the features.

# I. DATA IMPORTATION

In [1]:
import random
import numpy as np
from sklearn import svm
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
import pandas as pd
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import networkx as nx
import re
from sklearn.metrics import accuracy_score
from sknn.mlp import Layer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sknn.mlp import Classifier as MLP
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [2]:
info = pd.read_csv(
    "node_information.csv", 
    header= None, 
    names=["Id", "year", "title", "authors", "journal", "abstract"],
    sep=",",
    index_col = 0
)

In [3]:
X_train = pd.read_csv("training_set.txt", sep=" ", header=None)
X_test = pd.read_csv("testing_set.txt", sep=" ", header=None)
y_train = X_train[2]
X_train.drop([2], axis = 1, inplace = True)

# II. FEATURE ENGINEERING

This part includes :
- Feature Preprocess
- Topologic features (graph of the articles)
- Semantic features
- Attributes features 
- Topologic features (graph of the authors)

# II.1 Feature Preprocess

- list_authors is the list of authors in the papers. They are presented as follow : "Michalis Vazirgiannis, Xavier Le Pennec et al" becomes ["m. vazirgiannis", "x. pennec"]
- list_universities is the list where the authors are from

In [4]:
from authors_and_universities import authors_and_universities

list_authors, list_universities = authors_and_universities(info)
info['authors'] = list_authors
info['universities'] = list_universities

# II. 2. Topologic features

Creates a graph of the articles, then compute the following features : 
- Betweeness centrality
- Number of common neighbours
- Jaccard Coefficient
- Difference in inlinks coefficient
- Number of times "to" cites
- Same cluster

In [5]:
from create_topologic_features import make_graph, create_topologic_features

G = make_graph(X_train, y_train, X_test)  
X_train = create_topologic_features(X_train, G)
X_test = create_topologic_features(X_test, G)

# II.3. NLP features


For NLP features, we will:
### 1. Train a word 2 vec model
We concatenate the abstract and title texts as sentence inputs for training

In [9]:
from semantic_features import *


sentences = []  # Initialize an empty list of sentences
word_list_abstract=pd.Series(index=info.index,dtype=np.str)
word_list_title=pd.Series(index=info.index,dtype=np.str)

for idx in info.index:
    w_list_a = text_to_sentences(info.loc[idx,"abstract"], tokenizer, True)
    sentences += w_list_a
    word_list_abstract[idx]= w_list_a
    w_list_t = text_to_sentences(info.loc[idx,"title"], tokenizer, True)
    sentences += w_list_t
    word_list_title[idx]= w_list_t

num_features = 200
model = load_model(
    sentences, 
    train = False, 
    num_features = num_features, 
    min_word_count = 10, 
    num_workers = 4, 
    context = 10, 
    downsampling = 1e-3
)

[nltk_data] Downloading package punkt to /home/bat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
CPU times: user 11min 32s, sys: 16 s, total: 11min 48s
Wall time: 9min 12s


/home/bat/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


We use this model to transform the abstract and title into a list of word vectors and compute the centroid of those lists

In [ ]:
centroid_title = dict(zip(info.index, getAvgFeatureVecs(info.title.values, model, num_features)))
centroid_abstract = dict(zip(info.index, getAvgFeatureVecs(info.abstract.values, model, num_features)))

We also use it to make word clusters.

In [ ]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

# Pre-allocate an array for the training set bags of centroids (for speed)
bag_centroids_abstract = {}
#centroids_title = {}
# Transform the training set reviews into bags of centroids
for review, idx in zip(word_list_abstract, word_list_abstract.index):
    bag_centroids_abstract[idx] = create_bag_of_centroids(review[0], word_centroid_map)

We also compute a list of words from title and abstract to compute the Jaccard similarity.

In [10]:
w_l_title = info.title.apply(lambda x: text_to_wordlist(x, True))
w_l_abstract = info.abstract.apply(lambda x: text_to_wordlist(x, True))

We call a function that computes all these variables 

In [12]:
%%time
X_train = create_nlp_features(
    X_train, centroid_title, centroid_abstract, bag_centroids_abstract, w_l_title, w_l_abstract)
X_test = create_nlp_features(
    X_test,centroid_title, centroid_abstract, bag_centroids_abstract, w_l_title, w_l_abstract)

CPU times: user 15min 53s, sys: 2.28 s, total: 15min 56s
Wall time: 15min 59s


/home/bat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:31: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point


# II.4. Attribute features

- Difference in publication year
- Number of common authors
- Self-citation
- Same journal

In [7]:
%%time
from create_attribute_features import create_attribute_features
X_train = create_attribute_features(X_train,info)
X_test = create_attribute_features(X_test,info)

Wall time: 3min 15s


# II.5. Author Graph features

- TBC: AUTHORS NORMALIZED NUMBER SAME CLUSTER
- TBC: AUTHORS SLUTER JACCARD
- Authors betweeness centrality
- Authors common neighbors
- Authors Jaccard Coefficient
- Authors max difference in inlinks
- Authors sum difference in inlins
- Authors max of times to cited
- Authors sum of times to cited
- Authors median of times to cited

In [12]:
%%time 

from author_graph import make_graph_authors, create_topologic_features_authors

G_authors = make_graph_authors(X_train, y_train, info)
X_train = create_topologic_features_authors(X_train, G_authors, info, betweeness = True, common_neigh_and_jacc = True, inlinks = True)
X_test = create_topologic_features_authors(X_test, G_authors, info,  betweeness = True, common_neigh_and_jacc = True, inlinks = True)

creating cluster features
cluster features created
creating cluster features
cluster features created
Wall time: 7min 30s


# III. CLASSIFIERS

In [38]:
X_train_light = X_train.drop([0,1,"Number of times to cited","Authors  of times to cited"], axis = 1)
X_test_light = X_test.drop([0,1,"Number of times to cited","Authors  of times to cited"], axis = 1)

In [64]:
X_train_light2 = X_train.drop([0,1,"Number common neighbours","Self citation","Same journal","Authors max difference in inlinks",
                                   "Number of times to cited","Authors  of times to cited"], axis = 1)
X_test_light2 = X_test.drop([0,1,"Number common neighbours","Self citation","Same journal","Authors max difference in inlinks",
                             "Number of times to cited","Authors  of times to cited"], axis = 1)

# III.1. Random Forest

In [ ]:
%%time 
rfc = RandomForestClassifier(n_estimators = 400,n_jobs=3)
rfc.fit(X_train_light2, y_train)
pred_rfc = rfc.predict(X_test_light2)

In [ ]:
plt.figure(figsize=(15, 5))
X_columns = X_train_light2.columns
ordering = np.argsort(rfc.feature_importances_)[::-1]

importances = rfc.feature_importances_[ordering]
feature_names = X_columns[ordering]

x = np.arange(len(feature_names))
plt.bar(x, importances)
plt.xticks(x + 0.5, feature_names, rotation=90, fontsize=15);

# III.2. Neural Networks

In [27]:
scaler1=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn = MLP(
layers=[
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
           Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.05, batch_size = 100,dropout_rate =0.1,
        n_iter=100,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 25,
        debug = True,
        random_state = 42
        #    regularize = 'L2'
)

clfnn = Pipeline([
    ("scaler", scaler1),
    ('neural network', nn)
    ])

clfnn.fit(X_train.values, y_train.values)
pred = clfnn.predict(X_test)

0.976028087186


In [39]:
scaler1_light=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn_light = MLP(
layers=[
    Layer("Rectifier", units=50),
    Layer("Rectifier", units=50),
    Layer("Rectifier", units=50),
    Layer("Rectifier", units=40),
    Layer("Rectifier", units=40),
           Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.05, batch_size = 100,dropout_rate =0.1,
        n_iter=150,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 25,
        debug = False,
        random_state = 43
        #    regularize = 'L2'
)

clfnn_light = Pipeline([
    ("scaler", scaler1_light),
    ('neural network', nn_light)
    ])

clfnn_light.fit(X_train_light.values, y_train.values)
pred = clfnn_light.predict(X_test_light)

0.974494404658
CPU times: user 32min 36s, sys: 43.3 s, total: 33min 19s
Wall time: 17min 29s


In [ ]:

scaler1=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn2 = MLP(
layers=[
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
           Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.05, batch_size = 100,dropout_rate =0.1,
        n_iter=100,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 25,
        debug = True,
        #    regularize = 'L2'
)

clfnn2 = Pipeline([
    ("scaler", scaler1),
    ('neural network', nn2)
    ])

clfnn2.fit(X_train_light.values, y_train.values)
pred = clfnn2.predict(X_test_light)

In [ ]:

scaler1=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn = MLP(
layers=[
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
           Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.05, batch_size = 100,dropout_rate =0.1,
        n_iter=100,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 25,
        debug = True,
        #    regularize = 'L2'
)

clfnn = Pipeline([
    ("scaler", scaler1),
    ('neural network', nn)
    ])

clfnn.fit(X_train.values, y_train.values)
pred = clfnn.predict(X_test)

In [ ]:

scaler1=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn = MLP(
layers=[
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
    Layer("Rectifier", units=100),
           Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.05, batch_size = 100,dropout_rate =0.1,
        n_iter=100,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 25,
        debug = True,
        #    regularize = 'L2'
)

clfnn = Pipeline([
    ("scaler", scaler1),
    ('neural network', nn)
    ])

clfnn.fit(X_train.values, y_train.values)
pred = clfnn.predict(X_test)

# III.3. Gradient Boosting

In [ ]:
gbc = GradientBoostingClassifier(n_estimators = 400, max_depth = 5)
gbc.fit(X_train_light, y_train)
pred_gbc1 = gbc.predict(X_test_light)

In [ ]:
gbc = GradientBoostingClassifier(n_estimators = 400, max_depth = 7)
gbc.fit(X_train_light, y_train)
pred_gbc2 = gbc.predict(X_test_light)

# III.4. Extra Trees

In [ ]:
etc = ExtraTreesClassifier(n_estimators = 400,n_jobs=2)
etc.fit(X_train.drop([0,1], axis = 1), y_train)
pred_etc = etc.predict(X_test.drop([0,1], axis = 1))

# III.5. Logistic Regression

In [ ]:
clf =  Pipeline([
    ("scaler", MinMaxScaler()),
    ("regression",LogisticRegression(C=200000,max_iter = 1000))
    ])
clf.fit(X_train.drop([0,1], axis = 1), y_train)
pred_lr = clf.predict(X_test.drop([0,1], axis = 1))

# IV. ASSEMBLING

In [13]:
def rounds(predict):
    return [0 if pr<=0.5 else 1 for pr in predict]

def make_pred(y_pred_array, weights = None):
    if weights ==None:
        weights = [1 for i in y_pred_array]
    return np.array(rounds(1./sum(weights)*sum([weights[i]*y for i,y in enumerate(y_pred_array)])))

# V. SUBMISSION

In [ ]:
def make_submission(predicted_label, name = 'submit.csv'):
    submit_d = d = {'id' : pd.Series(np.arange(1,X_test.shape[0]+1).astype(int)),
                    'category' : pd.Series(predicted_label).astype(int)}
    submit = pd.DataFrame(submit_d)
    submit.to_csv(name,index=False)
    return submit